In [333]:
import numpy as np

In [334]:
def function_S(input):
    # input should be array
    # output array
    flattened_input = input.flatten()
    output = [[0,           -flattened_input[2],    flattened_input[1]],
              [flattened_input[2],  0,              -flattened_input[0]],
              [-flattened_input[1], flattened_input[0],     0]]
    return np.array(output)

In [335]:
def rodrigues_formula(quaternion):
    '''
    quaternion -> R_tile_bar
    input: [w,x,y,z]
    output R_tile_bar (rotation matrix)
    '''
    return np.eye(3) + 2*np.matmul(function_S(quaternion[1:]), (quaternion[0]*np.eye(3) + function_S(quaternion[1:])))


In [336]:
l = 3
k = 1
q = [10, 10, 10]
Q = np.diag(np.hstack([np.diag(q[2]*np.eye(3)), np.diag(q[1]*np.eye(3)), np.diag(q[0]*np.eye(3))]))
V = np.diag(np.hstack([np.diag(0.1*np.eye(3)), np.diag(np.eye(3))]))
P_0 = np.diag(np.hstack([np.diag(np.eye(3)), np.diag(100*np.eye(3))]))
#initial state
p_0 = np.array([[2.5, 0, 10]]).T
R_0 = np.eye(3)

#initial estimation
p_hat_0 = np.array([[-2, 4, 3]]).T
Lambda_bar_0 = np.array([np.sqrt(2)/2, np.sqrt(2)/2, 0, 0]).T  # quaternion: w, x, y, z
## R_hat = R x R_tile_bar
R_hat = np.matmul(R_0, rodrigues_formula(Lambda_bar_0))

In [359]:
time = np.linspace(0, 10, 9)
for t in time:
    p = np.transpose(np.array([[2.5+2.5*np.cos(0.4*t), 2.5*np.sin(0.4*t), 10]]))
    v = np.transpose(np.array([[-np.sin(0.4*t), np.cos(0.4*t), 0.6*t]]))
    omega = np.transpose(np.array([[0.1*np.sin(t), 0.4*np.cos(2*t), 0.6*t]]))

[ 0.    1.25  2.5   3.75  5.    6.25  7.5   8.75 10.  ]


In [338]:
# landmarks
z = np.array([[[0, 0, 0]], 
              [[5, 0, 0]],
              [[2.5, 2.5, 0]]])

In [339]:
A11 = -function_S(omega)
A12 = np.zeros((3,3))
A21 = np.zeros((3,3))
A22 = -function_S(omega)
A = np.vstack((np.hstack((A11, A12)), np.hstack((A21, A22))))

In [340]:
def function_Pi(input):
    '''
    Pi_x := I_3 - xx^T
    input: array
    output P_x
    '''
    return np.eye(3) - np.matmul(input, np.transpose(input))

In [341]:
def function_d(input_rot, input_p, input_z):
    '''
    calculate d_i(t) := R^T(t)(p(t) - z_i)/|p(t)-z_i|
    '''
    norm = (input_p - input_z)/np.linalg.norm(input_p - input_z)
    return np.matmul(np.transpose(input_rot), norm)

In [342]:
for landmark_idx in range(l):
    # S(R_hat.T x z)
    #TODO: remember to change the R_0 and p_0 below for gerenal
    first = function_Pi(function_d(R_0, p_0, np.transpose(z[landmark_idx])))
    second = function_S(np.matmul(np.transpose(R_hat), np.transpose(z[landmark_idx])))
    final = -np.matmul(first, second)
    C_landmark = np.hstack((final, first))
    if landmark_idx == 0:
        C = C_landmark
    else:
        C = np.vstack((C, C_landmark))

In [343]:
def add_bar(input_rot, input):
    return np.matmul(np.transpose(input_rot), input)

In [357]:
# observer equation

# omega_hat , p_bar_hat_dot
# omega
first_upper = omega
#TODO: change this for general (R_0)
p_bar_hat = add_bar(R_0, p_hat_0)

# -S(omega)p_bat_hat + v_bar
#TODO: change this for general (R_0)
first_lower = np.matmul(-function_S(omega), p_bar_hat) + add_bar(R_0, v)
first_part = np.vstack((first_upper, first_lower))

# omega_hat second part upper
final = np.transpose(np.array([[0, 0, 0]], dtype=np.float64))
for landmark_idx in range(l):
    #q*S(R_hat.T x z)
    first = q[landmark_idx]*function_S(np.matmul(np.transpose(R_hat), np.transpose(z[landmark_idx])))
    #Pi_d
    #TODO: remember to change the R_0 and p_0 below for gerenal
    second = function_Pi(function_d(R_0, p_0, np.transpose(z[landmark_idx])))
    #(p_bar_hat - R_hat.T x z)
    third = p_bar_hat - np.matmul(np.transpose(R_hat), np.transpose(z[landmark_idx]))
    final += np.matmul(first, np.matmul(second, third))

# omega_hat second part lower
final2 = np.transpose(np.array([[0, 0, 0]], dtype=np.float64))
for landmark_idx in range(l):
    #q*Pi_d
    #TODO: remember to change the R_0 and p_0 below for gerenal
    first = q[landmark_idx]*function_Pi(function_d(R_0, p_0, np.transpose(z[landmark_idx])))
    #(p_bar_hat - R_hat.T x z)
    second = p_bar_hat - np.matmul(np.transpose(R_hat), np.transpose(z[landmark_idx]))
    final2 += np.matmul(first, second)
second_part = np.vstack((final, final2))
#kP[]
#full second part 
second_part = k*np.matmul(P_0, second_part)

# Final
omega_hat_p_bar_hat_dot = first_part - second_part

In [346]:
# observer equation

R_hat_dot = np.matmul(R_hat, function_S(omega))

In [356]:
# observer equation

P_dot = np.matmul(A, P_0) + np.matmul(P_0, np.transpose(A)) - np.matmul(P_0, np.matmul(np.transpose(C), np.matmul(Q, np.matmul(C, P_0)))) + V